<a href="https://colab.research.google.com/github/ObliviousAI/enclave_workshops/blob/master/workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/Workshop_2_Synthetic_Data_from_Multiple_Private_Data_Sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop 2: Synthetic Data from Multiple Private Data Sources

This can all be done form your local machine, but for easy reproducibility we'll user a Google Colab.

### Step 1: Install the oblv proxy

In [ ]:
%%bash
# downlaod the OBLV CLI package for Linux 
wget https://oblv-cli-binary.s3.us-east-2.amazonaws.com/0.3.0/oblv-ccli-0.3.0-x86_64-unknown-linux-musl.zip
# unzip it
unzip oblv-ccli-0.3.0-x86_64-unknown-linux-musl.zip
# cd to the folder where binary is present
cd oblv-ccli-0.3.0-x86_64-unknown-linux-musl/
# update execute permissions for the binary
chmod +x $(pwd)/oblv
# create symlink
sudo ln -s $(pwd)/oblv /usr/local/bin/oblv

### Step 2.a: Create Keys for Alice and Bob (Skip if using example keys provided in the workshop)

In [ ]:
!oblv keygen --key-name alice --output /content/

In [ ]:
!oblv keygen --key-name bob --output /content/

### Step 2.b: Load in the public/private example keys from GitHub

In [ ]:
import base64
import requests

files = [
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/alice_private.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/alice_public.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/bob_private.cer",
    "workshops/2_Synthetic_Data_from_Multiple_Private_Data_Sources/bob_public.cer"
]

for file_name in files:
  url = 'https://raw.githubusercontent.com/ObliviousAI/enclave_workshops/master/'+file_name
  req = requests.get(url)
  if req.status_code == requests.codes.ok:
      file_to_save = file_name.split('/')[-1]

      with open('/content/' + file_to_save, 'wb') as file:
        file.write(req.content)
  else:
      print('Content was not found.')

### Step 3: Create a secure enclave connection from the localhost (in Google Colab) to the desired enclave

In [ ]:
%%script bash --bg
oblv connect \
--pcr0 43bd1391d6441fbbe437376fb95ed54061c296d42adf87dc21f5a1f20bc6333865ada371051f5ba098cb98b27037160b \
--pcr1 bcdf05fefccaa8e55bf2c8d6dee9e79bbff31e34bf28a99aa19e6b29c37ee80b214a414b7607236edf26fcb78654e63f \
--pcr2 0856cfa6d9935f092484c0aac705e3a5c9c7fcf79335e852cc5d9948d2265e82f0da89e7b757e65c62a9b34f7920e42d \
--private-key "/content/alice_private.cer" \
--public-key "/content/alice_public.cer" \
--url https://conso-appli-1ezgzeuvtvlbe-309102335.enclave.oblivious.ai \
--port 443 --lport 3030 

In [ ]:
%%script bash --bg

oblv connect \
--pcr0 43bd1391d6441fbbe437376fb95ed54061c296d42adf87dc21f5a1f20bc6333865ada371051f5ba098cb98b27037160b \
--pcr1 bcdf05fefccaa8e55bf2c8d6dee9e79bbff31e34bf28a99aa19e6b29c37ee80b214a414b7607236edf26fcb78654e63f \
--pcr2 0856cfa6d9935f092484c0aac705e3a5c9c7fcf79335e852cc5d9948d2265e82f0da89e7b757e65c62a9b34f7920e42d \
--private-key "/content/bob_private.cer" \
--public-key "/content/bob_public.cer" \
--url https://conso-appli-1ezgzeuvtvlbe-309102335.enclave.oblivious.ai \
--port 443 --lport 3031 

### Step 4: Interact with the enclave by sending traffic to the localhost on the port specified

In [ ]:
import requests
import json

ALICE_PROXY = "http://localhost:3030"
BOB_PROXY = "http://localhost:3031"

In [ ]:
r = requests.get(ALICE_PROXY+"/hello")

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/settings")

print(json.dumps(json.loads(r.content), indent=4))

In [ ]:
r = requests.get(ALICE_PROXY+"/call", params={"url": "https://example.com"})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/call", params={"url": "http://bad_example.com"})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/submit_value", params={"value": 5})

print(json.loads(r.content))

In [ ]:
r = requests.get(BOB_PROXY+"/submit_value", params={"value": 6})

print(json.loads(r.content))

In [ ]:
r = requests.get(ALICE_PROXY+"/compare")

print(json.loads(r.content))